In [1]:
from backend.dendron.out_go import process_file
from langchain.document_loaders import PDFPlumberLoader

# Define the path to the file, and other parameters
file_path = "/mnt/d/ai/cognimesh/output/katoAnticoagulationStrategiesExtracorporeal2021.pdf"
chunk_size = 1000
chunk_overlap = 50

# Now call the process_file function
process_file(path=file_path,
             loader_class=PDFPlumberLoader,
             chunk_size=chunk_size,
             chunk_overlap=chunk_overlap)



In [1]:
from backend.plugins.zotero_connect import ZoteroConnect
from backend.settings.backend_setting import store

# Create an instance of the plugin
zotero_plugin = ZoteroConnect()

# Assuming the 'store' object is your database connection from backend_setting
db_connection = store.connection

# Enable the plugin (if your system requires this step)
zotero_plugin.switch(enable=True)

# Set up the plugin, which includes table creation
zotero_plugin.setup(db_connection)

# You might want to catch any exceptions here and handle them appropriately
try:
    zotero_plugin.setup(db_connection)
except Exception as e:
    # Log the exception, alert the user, etc.
    print(f"An error occurred during ZoteroConnect plugin setup: {e}")

try:
    # Disable the plugin first
    zotero_plugin.switch(enable=False)
    
    # Then call the delete method to clean up resources
    zotero_plugin.delete()

except Exception as e:
    # Log the exception, alert the user, etc.
    print(f"An error occurred during ZoteroConnect plugin deletion: {e}")

2023-11-10 21:03:29,596 - backend.settings.backend_setting - INFO - OpenAI API key is valid.
2023-11-10 21:03:29,606 - zotero_connect - INFO - ZoteroConnect plugin has been enabled.
2023-11-10 21:03:29,607 - zotero_connect - INFO - Setting up ZoteroConnect plugin.
2023-11-10 21:03:29,606 - zotero_connect - INFO - ZoteroConnect plugin has been enabled.
2023-11-10 21:03:29,607 - zotero_connect - INFO - Setting up ZoteroConnect plugin.
2023-11-10 21:03:29,611 - zotero_connect - INFO - Creating zotero_articles and zotero_books tables.
2023-11-10 21:03:29,636 - zotero_connect - INFO - ZoteroConnect plugin has been enabled.
2023-11-10 21:03:29,637 - zotero_connect - INFO - ZoteroConnect plugin setup completed successfully.
2023-11-10 21:03:29,638 - zotero_connect - INFO - Setting up ZoteroConnect plugin.
2023-11-10 21:03:29,646 - zotero_connect - INFO - ZoteroConnect plugin has been enabled.
2023-11-10 21:03:29,646 - zotero_connect - INFO - ZoteroConnect plugin setup completed successfully.
